<H1><B><U><I>Determining Cluster of Tehsils in Delhi for Setting up Fashion Retail Stores</I></U></B></H1>

![Fashion Industry](https://akm-img-a-in.tosshub.com/indiatoday/images/story/201808/fashion_design_career.jpeg?1X_O.yI5zayM62fgH17uh1nFyD2BjdmL)

# 1) <B>INTRODUCTION</B>
<p>According to McKinsey and Company, India’s apparel market alone is estimated to be of worth \\$59.3 Billion by 2022, making it the sixth largest in the world, comparable to the United Kingdom’s (\\$65 billion) and Germany’s (\\$63.1 billion)[1]. The growth rate of fashion industry in India is driven by its increasingly growing population and increasing spending behaviour of people. Factors like literacy rate of people, population and gender also highly affect the success of a fashion brand/retail stores in a particular location[2]. This project attempts to find clusters of Tehsils in Delhi where an existing/new <i>Fashion Brand</i> can set up their retail stores.</p>

# 2) <B>DATA DESCRIPTION</B>
<p>Details about Population, Literacy Rate and Sex Ratio of people living in each Tehsil of each District of Delhi is collected from the website of [Census India](https://www.censusindia.co.in/states/delhi). FourSquare API is used to get location coordinates of each Tehsil, venues in the neighbourhood of each Tehsil, user ratings of those venues etc. This data will be used to define clusters of Tehsils in which a fashion retail store can be set up.</p>

# 3) <B>METHODOLOGY</B>
<p>First, let's import necessary dependencies:</p>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
from sklearn.cluster import KMeans
from pandas.plotting import table

print("All Libraries Loaded")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## 3.1. <i>Fetching Data</i>
<p>We will import Tehsil data of Delhi as dataframe:</p>

In [ ]:
tehsil = pd.read_excel('/kaggle/input/Tehsil Data of Delhi.xlsx')
tehsil.columns = ['Neighborhood', 'Borough', 'Population', 'Literacy', 'Sex Ratio', 'Postal Code'] #Neighborhood: Tehsil of Delhi, Borough: District of Delhi in which the Tehsil is situated, Postal Code: NIC Tehsil Code
tehsil.head()

In [ ]:
tehsil.shape

In [ ]:
tehsil['Latitude'] = ""
tehsil['Longitude'] = ""
tehsil.head()

<p>Let's see if we can get coordinates of Chanakyapuri. Here, we will use 'geocoder' class to get coordinates 

In [ ]:
query = '{}, Chanakyapuri, New Delhi'
g = geocoder.arcgis(query.format(2085))
g.latlng

Now, we will apply the above method of getting coordinates to all the Tehsils/Neighborhoods in the dataframe.

In [ ]:
for i in range(tehsil.shape[0]):
    code = tehsil.loc[i, 'Postal Code']
    address = tehsil.loc[i, 'Neighborhood'] + ', ' + tehsil.loc[i, 'Borough']
    query = '{}, ' + address
    g = geocoder.arcgis(query.format(code))
    tehsil.loc[i, 'Latitude'] = g.latlng[0]
    tehsil.loc[i, 'Longitude'] = g.latlng[1]
tehsil.head()

In [ ]:
tehsil.Literacy = tehsil.Literacy * 100

## 3.2 <i>Visualizing Delhi on Map</i> 

In [ ]:
# Longitude and Latitude of Delhi
latitude = 28.7041
longitude = 77.1025

# create map of Delhi using latitude and longitude values
map_on = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tehsil['Latitude'], tehsil['Longitude'], tehsil['Borough'], tehsil['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_on)  
    
map_on

## 3.3 <i>Exploring Population, Literacy Rate and Sex Ratio by Districts of Delhi</i>

In [ ]:
ratio = tehsil.groupby('Borough')['Sex Ratio'].mean()
pop = tehsil.groupby('Borough')['Population'].mean()
literacy = tehsil.groupby('Borough')['Literacy'].mean()

plt.style.use('ggplot')
fig = plt.figure(figsize = (20, 20))
ax_0 = fig.add_subplot(3, 1, 1) # add subplot 1 
ax_1 = fig.add_subplot(3, 1, 2) # add subplot 2
ax_2 = fig.add_subplot(3, 1, 3) # add subplot 2

#Subplot 1: Population
pop.plot.bar(sharex = True, ax = ax_0, color=['steelblue', 'steelblue', 'steelblue', 'steelblue', 'steelblue', 'crimson', 'steelblue', 'steelblue', 'rebeccapurple'])
ax_0.set_ylabel('Population', fontsize = 23)
ax_0.set_xlabel('Districts of Delhi', fontsize = 23)
ax_0.tick_params(axis='both', which='major', labelsize=20)
ax_0.set_title("Average Population in Each District of Delhi", fontdict = {'fontsize' : 20}, y = 1.04)

#Subplot 2: Sex Ratio
ratio.plot.bar(sharex = True, ax = ax_1, color=['gray', 'gray', 'lightseagreen', 'lightseagreen', 'gray', 'lightseagreen', 'lightseagreen', 'lightseagreen', 'rebeccapurple'])
ax_1.set_ylabel('Sex Ratio', fontsize = 23)
ax_1.set_xlabel('Districts of Delhi', fontsize = 23)
ax_1.tick_params(axis='both', which='major', labelsize=20)
ax_1.set_title("Average Sex Ratio in Each District of Delhi", fontdict = {'fontsize' : 20}, y = 1.04)

#Subplot 3: Literacy
literacy.plot.bar(sharex = True, ax = ax_2, color=['goldenrod', 'goldenrod', 'forestgreen', 'goldenrod', 'goldenrod', 'goldenrod', 'goldenrod', 'goldenrod', 'rebeccapurple'])
ax_2.set_ylabel('Literacy', fontsize = 23)
ax_2.set_xlabel('Districts of Delhi', fontsize = 23)
ax_2.tick_params(axis='both', which='major', labelsize=20)
ax_2.set_title("Average Literacy in Each District of Delhi", fontdict = {'fontsize' : 20}, y = 1.04)

fig.tight_layout(pad = 3.0)
plt.savefig('Visualizations by Districts of Delhi.png')
plt.show()

<b><u>Observations:</u></b>
<li> North West Delhi has highest population followed by South Delhi and then, West Delhi.
<li> Central Delhi, East Delhi and North East Delhi have highest Sex Ratio
<li> New Delhi has highest Literacy Rate followed by East Delhi
<br><br>
<b><u>Inference:</u></b><br>
<br><p>From The Above Plots, We can infer Tehsils in West Delhi can be considered to be good locations for fashion store as they have good population size, good sex ratio and descent literacy rate. But for those fashion stores that are aiming for niche market, Tehsils in East Delhi can be better locations to target as it has low population size, high literacy rate and moderate sex ratio.</p>



## 3.4 <i>Exploring Population, Literacy Rate and Sex Ratio by Tehsils of Delhi</i>

In [ ]:
plt.style.available

In [ ]:
ratio = tehsil.groupby('Neighborhood')['Sex Ratio'].mean().sort_values(ascending = False).head(7)
pop = tehsil.groupby('Neighborhood')['Population'].mean().sort_values(ascending = False).head(7)
literacy = tehsil.groupby('Neighborhood')['Literacy'].mean().sort_values(ascending = False).head(7)

plt.style.use('_classic_test_patch')
fig = plt.figure(figsize = (20, 30))
ax_0 = fig.add_subplot(3, 1, 1) # add subplot 1 
ax_1 = fig.add_subplot(3, 1, 2) # add subplot 2
ax_2 = fig.add_subplot(3, 1, 3) # add subplot 2

#Subplot 1: Population
pop.plot.bar(ax = ax_0, color='crimson')
ax_0.set_ylabel('Population', fontsize = 23)
ax_0.set_xlabel('Tehsil of Delhi', fontsize = 23)
ax_0.tick_params(axis='both', which='major', labelsize=20)
ax_0.set_title("Average Population in Each Tehsil of Delhi", fontdict = {'fontsize' : 20}, y = 1.04)

#Subplot 2: Sex Ratio
ratio.plot.bar(ax = ax_1, color='lightseagreen')
ax_1.set_ylabel('Sex Ratio', fontsize = 23)
ax_1.set_xlabel('Tehsil of Delhi', fontsize = 23)
ax_1.tick_params(axis='both', which='major', labelsize=20)
ax_1.set_title("Average Sex Ratio in Each Tehsil of Delhi", fontdict = {'fontsize' : 20}, y = 1.04)

#Subplot 3: Literacy
literacy.plot.bar(ax = ax_2, color='steelblue')
ax_2.set_ylabel('Literacy', fontsize = 23)
ax_2.set_xlabel('Tehsil of Delhi', fontsize = 23)
ax_2.tick_params(axis='both', which='major', labelsize=20)
ax_2.set_title("Average Literacy in Each Tehsil of Delhi", fontdict = {'fontsize' : 20}, y = 1.04)

fig.tight_layout(pad = 3.0)
plt.savefig('Visualizations by Tehsils of Delhi.png')
plt.show()

* Karol Bagh has highest Literacy rate and Sex ratio. But its population size is not even in top seven.
* Tehsils which have high population do not have high Sex Ratio and Literacy rate. 

## <i>3.5 Exploring Venues in the Tehsils</i>

In [ ]:
tehsil.head()

First, lets explore top 100 venues in Connaught Place. Below we will define credentials of FourSquare API to fetch venues in any Tehsil.

In [ ]:
CLIENT_ID = 'SMW0KBR1QLGVOEBX3HFOQFVCABUB5HXWFTUZZCZMHFD5PVTW' 
CLIENT_SECRET = '3KAMZFVCZ15XQ04SVTKVDMVTEFQJASZNQRSVTUUNUZHX4PDZ'
VERSION = '20200406'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
neighborhood_latitude = tehsil.loc[1, 'Latitude'] # Tehsil latitude value
neighborhood_longitude = tehsil.loc[1, 'Longitude'] # Tehsil longitude value

neighborhood_name = tehsil.loc[1, 'Neighborhood']# Tehsil name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Then, We will create GET request URL:

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    1020, 
    100)
url

After this, we will fetch URL from FourSquare API in JSON format and convert it into dataframe.

In [ ]:
results = requests.get(url).json()

# Defining function to extract categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

Now, we will get venues in all Tehsils of Delhi.

In [ ]:
# The following function fetches venues from foursquare api:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            1020, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
tehsil_venues = getNearbyVenues(names=tehsil['Neighborhood'],
                                   latitudes=tehsil['Latitude'],
                                   longitudes=tehsil['Longitude']
                                  )

In [ ]:
print(tehsil_venues.shape)
tehsil_venues.head(10)

249 venues in all tehsils are returned by FourSquare API as shown above. The following code shows the number of venues in each tehsil:

In [ ]:
df = pd.DataFrame(tehsil_venues.groupby('Neighborhood').count())
df

Let's find out how many unique categories can be curated from all the returned venues:

In [ ]:
print('There are {} uniques categories.'.format(len(tehsil_venues['Venue Category'].unique())))

## 3.6 <i>Analyzing Each Neighborhood</i>

<p>First, we will get frequencies of occurrence of each venue category.</p>

In [ ]:
# one hot encoding
tehsil_onehot = pd.get_dummies(tehsil_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tehsil_onehot['Neighborhood'] = tehsil_venues['Neighborhood'] 
tehsil_onehot.head()

In [ ]:
tehsil_onehot.shape

<p>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:</p>

In [ ]:
tehsil_grouped = tehsil_onehot.groupby('Neighborhood').mean().reset_index()
tehsil_grouped

In [ ]:
tehsil_grouped.shape

<p>Let's print each neighborhood along with the top 5 most common venues:</p>

In [ ]:
num_top_venues = 10

for hood in tehsil_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tehsil_grouped[tehsil_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

<p>Now, We will convert it into dataframe.</p>

In [ ]:
# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tehsil_venues_sorted = pd.DataFrame(columns=columns)
tehsil_venues_sorted['Neighborhood'] = tehsil_grouped['Neighborhood']

for ind in np.arange(tehsil_grouped.shape[0]):
    tehsil_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tehsil_grouped.iloc[ind, :], num_top_venues)

In [ ]:
df = tehsil_venues_sorted.head(10)
df

## 3.7 <i>Clustering Tehsils</i>

<p>Now, we will group tehsils into cluster and identify potential ones among them to target for. For this, we will run k-means algorithm to form clusters among tehsil (here, we are forming 5 clusters).</p>

In [ ]:
# set number of clusters
kclusters = 8

tehsil_grouped_clustering = tehsil_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tehsil_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
tehsil_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tehsil_merged = tehsil

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tehsil_merged = tehsil_merged.join(tehsil_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tehsil_merged.head() # check the last columns!

In [ ]:
tehsil_merged.dropna(inplace=True)

<p>Finally, Lets visualize the clusters:</p>

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tehsil_merged['Latitude'], tehsil_merged['Longitude'], tehsil_merged['Neighborhood'], tehsil_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3.8 <i>Examining Clusters</i>

### <B>Cluster 1: East Delhi and North East Delhi</B>

In [ ]:
cluster1 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 0, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster1

<p>In cluster 1, tehsils that have Pizza Place, Indian Restaurant and, Shop and Service are clubbed together.</p>

### <B> Cluster 2: New Delhi, Central Delhi, North Delhi, East Delhi, South-West Delhi, West Delhi, South Delhi and North-West Delhi</B>

In [ ]:
cluster2 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 1, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster2

<p>In cluster 2, Tehsils with Indian/Chinese/Italian Restaurant, cafe/coffee shop, hotel and pubs are clubbed together. This is the biggest cluster found through k-means algorithm.</p> 

### <b>Cluster 3: North-East Delhi and North-West Delhi

In [ ]:
cluster3 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 2, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster3

<p>In Cluster 3, tehsils with ATM, Electronic Store, Event Space, Falafel Restaurant and Farmers Market are clubbed together.</p>

### <b>Cluster 4: South-West Delhi</b>

In [ ]:
cluster4 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 3, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster4

<p>This Cluster has almost same venue categories as in cluster 3. So, Cluster 4 and Cluster 3 can be clubbed together.</p>

### <b>Cluster 5: North Delhi, West Delhi and South Delhi</b>

In [ ]:
cluster5 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 4, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster5

<p>In Cluster 5, Tehsils with Pizza Place and Fast Food Restaurant are clubbed together.</p>

### <b>Cluster 6, 7, 8:</b>

In [ ]:
cluster6 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 5, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster6

In [ ]:
cluster7 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 6, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster7

In [ ]:
cluster8 = tehsil_merged.loc[tehsil_merged['Cluster Labels'] == 7, tehsil_merged.columns[[1] + list(range(5, tehsil_merged.shape[1]))]]
cluster8

<p>The Clusters 6, 7 and 8 are almost similar. So, They can be clubbed together. Hence, There are 5 clusters in total found after clubbing similar clusters.</p>

## 3.9 <i>Finding and Visualizing Prospective Tehsils Based on a Criteria</i>
<p>Suppose a client wants to set up fashion retail stores at Tehsils in Delhi which has Indian Restaurant, Cafe and Pubs. I've used a violin plot from the seaborn library - it is a great way to visualise frequency distribution datasets, they display a density estimation of the underlying distribution.</p>

In [ ]:
import seaborn as sns

fig = plt.figure(figsize=(65,65))
sns.set(font_scale=2.6)

ax = plt.subplot(3,1,1)
sns.violinplot(x="Neighborhood", y="Indian Restaurant", data=tehsil_onehot, cut=0);
plt.xlabel("")
plt.ylabel("Indian Restaurant", fontsize=50)
plt.xticks(rotation=45)

ax = plt.subplot(3,1,2)
sns.violinplot(x="Neighborhood", y="Café", data=tehsil_onehot, cut=0);
plt.xlabel("")
plt.ylabel("Cafe", fontsize=50)
plt.xticks(rotation=45)

plt.subplot(3,1,3)
sns.violinplot(x="Neighborhood", y="Pub", data=tehsil_onehot, cut=0);
plt.ylabel("Pub", fontsize=50)
plt.xticks(rotation=45)
plt.xlabel("Tehsils of Delhi", fontsize=50)
ax.text(-1.0, 3.1, 'Frequency distribution for the venue categories for each neighborhood', fontsize=60)
fig.tight_layout()
plt.savefig ("Distribution_Frequency_Venues_3_categories.png", dpi=300)
plt.show()

<p>Based on above violin plots, <b>Chanakya Puri</b>, <b>Connaught Place</b>, <b>Parliament Street</b>, <b>Vasant Vihar</b> and <b>Rajouri Garden</b>. Now, let's see in which cluster these locations lie.</p> 

In [ ]:
tehsil_select = tehsil_merged.loc[tehsil_merged.Neighborhood.isin(['Chanakya Puri', 'Connaught Place', 'Parliament Street', 'Vasant Vihar', 'Rajouri Garden'])]
tehsil_select

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tehsil_select['Latitude'], tehsil_select['Longitude'], tehsil_select['Neighborhood'], tehsil_select['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<p>As West Delhi has good population size, good literacy rate and descent gender ratio, we can narrow down our choice for prospective locations for a fashion retail store to Rajouri Garden as it is in West Delhi District.</p>

# 4) <b>CONCLUSION</b>
<p>We conclude that based on certain parameters or criteria, we can select one or two clusters of Tehsils of Delhi through the above methodology for the establishment of a retail store.</p>
<p>There may be other ways this analysis could have been performed based on different methodolgy, different algorithm and perhaps different data sources. But I chose this method as the dataset related to tehsils in Delhi is readily available through Census India, and also, K-Means algorithm is easily able to be applied on this dataset.</p>

# 5) <b>LIMITATIONS</b>
1. Area of each Tehsil can also be used for analysis.
2. Data related to centennials and millennials can be employed for decision-making and, thus can be used for further analysis.
3. Fashion Brands famous or trending in a particular tehsil can also be helpful to decide better locations for establishment of fashion retail stores.

# 6) <b>REFERENCES</b>
[1]	Arvind Singhal, “India’s Fashion and Retail Sector: An Overview”, Brand Growth Inspiration, 12th April 2018<br>
[2]	Amed, Imran et. al., “How India’s Ascent could change the Fashion Industry”, McKinsey & Company – Our Insight, February 2019 
